# ManimCE Descomplicado

## Módulo 2 - Grupos, transformações e updaters

In [2]:
from manim import *

Manim Community v0.17.3

### 2.1 Agrupando Objetos e Animações

Às vezes, será mais útil trabalhar com objetos agrupados do que com os objetos individuais. Podemos reunir um grupo e aplicar transformações sobre todos os objetos contidos nele de uma só vez. Para esse propósito, a classe `VGroup` reúne diversos _MObjects_ num grupo, de modo que as mudanças aplicadas ao grupo são transmitidas a todos os elementos contidos nele, isto é, transformar, dimensionar, organizar e animar objetos.

In [2]:
class VGroupEx(Scene):
    def construct(self):

        s1 = Square(color = RED)
        s2 = Square(color = GREEN)
        s3 = Square(color = BLUE)

        square_group = VGroup(s1, s2, s3)

        square_group.arrange_in_grid(1, 3)

        self.play(
            Write(square_group)
        )
        
        self.play(
            square_group[1].animate.scale(1.5).shift(DOWN),
            square_group[2].animate.shift(RIGHT),
            square_group[0].animate.shift(LEFT)
        )

        colors = [RED, GREEN, BLUE]

        sq_anim = AnimationGroup(square_group[0].animate.set_fill(colors[0], 1),
                                 square_group[1].animate.set_fill(colors[1], 1),
                                 square_group[2].animate.set_fill(colors[2], 1), 
                                 lag_ratio = .2)

        self.play(sq_anim)

        self.wait()

In [3]:
%manim -qm -v warning VGroupEx

### 2.2 Organizando Objetos - Arrange e Arrange in Grid

A classe `VGroup` contém funções de organização dos arranjos de objetos. É uma maneira simples de ordenar os elementos do grupo em tela, de modo que eles fiquem próximos um do outro numa direção especificada. 

A função `arrange` acompanha o `VGroup` e organiza os elementos de acordo com alguma direção (DOWN, UP, RIGHT, LEFT, ou uma combinação destes).

Outra função muito comum e mais geral é o `arrange_in_grid` que organiza os elementos em uma grade, ou matriz de objetos. Essa função tenta colocar os objetos numa grade de acordo com as dimensões especificadas como argumento, como número de linhas e colunas. Por exemplo, se tenho um grupo de 4 quadrados, posso querer arranjá-los numa grade de 2x2. Para isso, utilizo `squares.arrange_in_grid(2, 2)`.

In [4]:
from random import uniform

class ArrangeEx(Scene):
    def construct(self):

        circles = VGroup(
            *[Circle(radius = .1).scale(uniform(.4, 3)).shift(UP * uniform(-3, 3) + RIGHT * uniform(-4, 4))
              for _ in range(12)]
        )

        self.play(FadeIn(circles))

        # Sem especificar direção, o método organiza da esquerda para a direita
        self.play(circles.animate.arrange())

        # Especificando direções
        self.play(circles.animate.arrange(DOWN, buff = .2))
        self.play(circles.animate.arrange(LEFT + UP, buff = .2))

        self.wait(.5)

        self.play(Unwrite(circles))

In [5]:
%manim -qm -v warning ArrangeEx

O argumento _buff_ define a separação entre os elementos consecutivos. Quanto menor for o _buff_, mais próximos os objetos estarão um do outro na direção que foi especificada.

Vamos agora construir uma cena com `arrange_in_grid`, organizando os objetos numa grade ou matriz.

In [8]:
class ArrangeInGridEx(Scene):
    def construct(self):

        '''
        Criar conjunto de quadrados agrupados com VGroup.
        Utilizo o .scale() para ajustar os tamanhos, adi-
        ciono texto nos centros dos quadrados com .add(),
        e espalho os quadrados ao longo da tela com .shift().
        '''

        squares = VGroup(
            *[Square()
              .scale(.5)
              .add(Tex(str(i + 1)).scale(.8))
              .shift(UP * uniform(-5, 5) + LEFT * uniform(-5, 5))
              for i in range(25)]
        )

        self.play(
            Write(squares, lag_ratio = 0)
        )

        '''
        O método arrange_in_grid acompanha o objeto e, no
        contexto da cena, é animado junto com o objeto.
        Novamente, o buff é a separação dos elementos e,
        neste caso, pode ser uma tupla especificando o espa-
        çamento horizontal e vertical.
        '''

        self.play(
            squares.animate.arrange_in_grid(5, 5, buff = (.2, .2))
            )

        self.wait(.5)

        self.play(
            Unwrite(squares, lag_ratio = 0)
        )

In [9]:
%manim -qm -v warning ArrangeInGridEx

No código acima, fizemos nosso procedimento padrão de criação de cenas. Você já deve ter percebido agora o quão útil é a utilização da classe **VGroup()** para agrupar objetos de um mesmo tipo e manipulá-los antes mesmo de qualquer animação. Fizemos três manipulações com os quadrados que criamos. 

Primeiramente, utilizando o `.scale()` ajustamos o tamanho de cada um. 

Em seguida, com `add()` eu quis adicionar os números para identificar cada quadrado, de modo que cada número fosse iterado dentro da list comprehension do próprio VGroup. 

Por último, o `shift()` foi utilizado para espalhar os quadrados pela cena, embaralhando-os antes de organizá-los. Para isso, multipliquei as direções com uma distribuição **uniforme**, que gera números aleatórios dentro de um intervalo.

A aplicação do **arrange_in_grid** é bem direta. Dentro do contexto do `self.play()`, o método sempre acompanhará o objeto e a sintaxe `animate`. Os argumentos consistem do formato ou _shape_ da matriz em que quero organizar os objetos. No caso, como temos 25 quadrados, o _shape_ $(x, y)$ deve ser tal que a multiplicação de x com y resulte em 25. O argumento _buff_ é o distanciamento dos elementos, que pode ser tomado como um _float_ individual, ou como uma tupla de _floats_, especificando os espaçamentos horizontal e vertical.

#### 2.3 Adicionando e removendo elementos

Podemos adicionar ou remover elementos de cena com os métodos `.add()` e `.remove()`. Também podemos manipular a ordem dos objetos na cena:

- `self.bring_to_front(MObject)`: traz o objeto para frente de outros objetos em cena.
- `self.bring_to_back(MObject)`: leva o objeto para trás.

In [3]:
class AddRemoveEx(Scene):
    def construct(self):

        square = Square(color = GREEN, fill_opacity = 1)
        triangle = Triangle(color = BLUE, fill_opacity = 1).scale(.5).move_to(square)

        self.play(Write(square))        

        # Adiciona o triângulo atrás do quadrado
        self.bring_to_back(triangle)
        self.play(square.animate.shift(LEFT * 2))

        # Adicionar um círculo à cena e movê-lo para a posição atual do quadrado
        circle = Circle(color = RED, fill_opacity = 1).scale(.5).move_to(square)

        # Levar o círculo para trás na cena 
        self.bring_to_back(circle) 
        self.play(square.animate.shift(RIGHT * 2))

        square2 = Square(color = WHITE, fill_opacity = 1).scale(.5).move_to(square)

        # Remover o triângulo da cena
        self.remove(triangle)

        # Adiciona um segundo quadrado na frente do outro quadrado 
        # (animação feia, mas só para mostrar funcionalidade)
        self.bring_to_front(square2)

        self.play(square.animate.shift(RIGHT * 2))

In [4]:
%manim -qm -v warning AddRemoveEx

Os objetos em cena são renderizados de acordo com a ordem em que aparecem no código, de maneira bem intuitiva. O primeiro objeto aparece na tela e, se outro objeto for criado depois dele, ele será criado em cima desse primeiro. Contudo, podemos manipular essa ordem pelo `z_index` do objeto (que nada tem a ver com o eixo coordenado).

In [49]:
class zIndexEx(Scene):
    def construct(self):

        c1 = Circle(color = RED, fill_opacity = 1).shift(RIGHT)
        c2 = Circle(color = BLUE, fill_opacity = 1)
        c3 = Circle(color = GREEN, fill_opacity = 1).shift(LEFT)

        tex_c1 = Tex('c1').next_to(c1, UP)
        tex_c2 = Tex('c2').next_to(c2, UP)
        tex_c3 = Tex('c3').next_to(c3, UP)

        texts = VGroup(tex_c1, tex_c2, tex_c3)
        title = Tex('Ordem dos objetos').next_to(texts, UP)

        # c3 na frente pela ordem de criação
        self.add(c1, c2, c3, tex_c1.set_opacity(.5), tex_c2.set_opacity(.5), tex_c3.set_opacity(1), title)

        self.wait(1.5)

        # c2 na frente dos outros objetos: c2(z = 1), c3 e c1 (z = 0)
        c2.set_z_index(1)

        self.add(tex_c1.set_opacity(.5), tex_c2.set_opacity(1), tex_c3.set_opacity(.5))

        self.wait(1.5)

        # c1 na frente dos outros objetos: c1 (z = 2), c2 (z = 1), c3 (z = 0)
        c1.set_z_index(2)

        self.add(tex_c1.set_opacity(1), tex_c2.set_opacity(.5), tex_c3.set_opacity(.5))

        self.wait(1.5)

In [50]:
%manim -qm -v warning zIndexEx